# Data selection and preprocessing:

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [29]:
ratings = pd.read_csv('ratings.csv')

In [19]:
def train_test_split_user(ratings, test_size=0.2):
    train_list = []
    test_list = []

    # Get all unique users
    for user_id in ratings['userId'].unique():
        # Filter the dataset to include only rows corresponding to curr user
        user_data = ratings[ratings['userId'] == user_id]
        # Check if user has rated more than 5 movies to meaningfully split data into train and test
        # Ex. user with 10 ratings -> 8 train, 2 test, but user with 3 ratings -> 2 train, 1 test
            # In the latter case, we would not have enough data to train the model
        # If user has rated more than 5 movies, split the data into train and test
        # Else, include all data in train
        if len(user_data) >= 5:
            train_data, test_data = train_test_split(user_data, test_size=test_size, random_state=42)
            train_list.append(train_data)
            test_list.append(test_data)
        else:
            train_list.append(user_data)
    
    # Combine training and testing data for all users into train and test
    train = pd.concat(train_list)
    test = pd.concat(test_list)
    return train, test

train_data, test_data = train_test_split_user(ratings)
print(train_data.head())
# We don't need timestamp column
train_data = train_data.drop(columns=['timestamp'])
test_data = test_data.drop(columns=['timestamp'])

     userId  movieId  rating  timestamp
55        1     1031     5.0  964982653
230       1     4006     4.0  964982903
69        1     1197     5.0  964981872
168       1     2596     5.0  964981144
109       1     1777     4.0  964981230


In [21]:
def quantile_normalize(ratings):
    # Group by userId
    normalized_ratings = ratings.groupby('userId')['rating'].transform(lambda x: quantile_scale(x))
    ratings['rating'] = normalized_ratings
    return ratings

def quantile_scale(ratings):
    # Sort ratings and assign ranks
    sorted_ratings = np.sort(ratings)
    ranks = np.argsort(np.argsort(ratings))  # Get ranks of the original ratings
    # Assign the sorted values to original ranks (quantile transformation)
    return np.interp(ratings, sorted_ratings, sorted_ratings)
train_data = quantile_normalize(train_data)

In [22]:
train_data['weight'] = train_data.groupby('movieId')['movieId'].transform('count')
train_data['weight'] = train_data['weight'] / train_data['weight'].sum()  # Normalize weights
train_data = train_data.sample(frac=1, weights=train_data['weight'], random_state=42)

In [23]:
class MF():

    def __init__(self, R, K, alpha, beta, iterations):
        self.R = R
        self.num_users, self.num_movies = R.shape
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations

    def train(self):
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_movies, self.K))

        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_movies)
        self.b = np.mean(self.R[np.where(self.R != 0)])

        self.samples = [
            (i, j, self.R[i, j])
            for i in range(self.num_users)
            for j in range(self.num_movies)
            if self.R[i, j] > 0
        ]

        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            print("Iteration: %d ; mae = %.4f ; rmse = %.4f" % (i+1, self.mae(), self.rmse()))
          
    def mae(self):
        xs, ys = self.R.nonzero() 
        predicted = self.full_matrix()  
        error = 0
        
        for x, y in zip(xs, ys):
            error += abs(self.R[x, y] - predicted[x, y])  
        
        return error / len(xs)
    
    def rmse(self):
        xs, ys = self.R.nonzero()
        predicted = self.full_matrix()
        error = 0
        for x, y in zip(xs, ys):
            error += pow(self.R[x, y] - predicted[x, y], 2)

        return np.sqrt(error/len(xs))

    def sgd(self):
        for i, j, r in self.samples:
            prediction = self.get_rating(i, j)
            e = (r - prediction)

            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_i[j] += self.alpha * (e - self.beta * self.b_i[j])

            P_i = self.P[i, :].copy()
            Q_j = self.Q[j, :].copy()

            self.P[i, :] += self.alpha * (e * Q_j - self.beta * P_i)
            self.Q[j, :] += self.alpha * (e * P_i - self.beta * Q_j)

    def get_rating(self, i, j):
        m_b = self.b_i[j] if j < len(self.b_i) else 0
        l_i = self.Q[j, :].T if j < len(self.Q) else np.mean(self.Q, axis=0) 
        prediction = self.b + self.b_u[i] + m_b + self.P[i, :].dot(l_i)
        return prediction

    def full_matrix(self):
        return self.b + self.b_u[:,np.newaxis] + self.b_i[np.newaxis:,] + self.P.dot(self.Q.T)


In [24]:
user_ids = train_data['userId'].unique()  
movie_ids = train_data['movieId'].unique()  

user_to_idx = {user_id: idx for idx, user_id in enumerate(user_ids)}
movie_to_idx = {movie_id: idx for idx, movie_id in enumerate(movie_ids)}

R = np.zeros((len(user_ids), len(movie_ids)))

for row in train_data.itertuples():
    user_idx = user_to_idx[row.userId]
    movie_idx = movie_to_idx[row.movieId]
    R[user_idx, movie_idx] = row.rating

mf = MF(R, K=50, alpha=0.01, beta=0.05, iterations=70)
# Iteration: 178 ; mae = 0.1979 ; rmse = 0.2732

mf.train()

Iteration: 1 ; mae = 0.7012 ; rmse = 0.8992
Iteration: 2 ; mae = 0.6778 ; rmse = 0.8752
Iteration: 3 ; mae = 0.6664 ; rmse = 0.8621
Iteration: 4 ; mae = 0.6572 ; rmse = 0.8532
Iteration: 5 ; mae = 0.6520 ; rmse = 0.8459
Iteration: 6 ; mae = 0.6473 ; rmse = 0.8404
Iteration: 7 ; mae = 0.6439 ; rmse = 0.8356
Iteration: 8 ; mae = 0.6405 ; rmse = 0.8311
Iteration: 9 ; mae = 0.6370 ; rmse = 0.8272
Iteration: 10 ; mae = 0.6338 ; rmse = 0.8238
Iteration: 11 ; mae = 0.6320 ; rmse = 0.8206
Iteration: 12 ; mae = 0.6286 ; rmse = 0.8174
Iteration: 13 ; mae = 0.6259 ; rmse = 0.8138
Iteration: 14 ; mae = 0.6239 ; rmse = 0.8106
Iteration: 15 ; mae = 0.6214 ; rmse = 0.8068
Iteration: 16 ; mae = 0.6181 ; rmse = 0.8026
Iteration: 17 ; mae = 0.6144 ; rmse = 0.7981
Iteration: 18 ; mae = 0.6104 ; rmse = 0.7926
Iteration: 19 ; mae = 0.6067 ; rmse = 0.7867
Iteration: 20 ; mae = 0.6018 ; rmse = 0.7798
Iteration: 21 ; mae = 0.5955 ; rmse = 0.7715
Iteration: 22 ; mae = 0.5890 ; rmse = 0.7631
Iteration: 23 ; mae

In [30]:
def test_model(mf_model, test_data, user_to_idx, movie_to_idx):
    squared_error = 0
    absolute_error = 0
    n = len(test_data)

    for _, row in test_data.iterrows():
        uid = int(row['userId'])
        mid = int(row['movieId'])
             
        user_id = int(user_to_idx[uid])
        movie_id = int(movie_to_idx[mid]) if mid in movie_to_idx else len(movie_to_idx)
        actual_rating = row['rating']
        
        predicted_rating = mf_model.get_rating(user_id, movie_id)
        
        squared_error += (actual_rating - predicted_rating) ** 2
        
        absolute_error += abs(actual_rating - predicted_rating)
    
    rmse = np.sqrt(squared_error / n)
    mae = absolute_error / n

    return rmse, mae

In [31]:
rmse, mae = test_model(mf, test_data, user_to_idx, movie_to_idx)
print("Test RMSE:", rmse)
print("Test MAE:", mae)

Test RMSE: 0.8611403600424232
Test MAE: 0.6592161081145493


In [ ]:
def unwatched_movies(ratings):
    all_movie_ids = set(ratings['movieId'])
    user_to_unwatched_movies = {}

    for user_id in ratings['userId'].unique():
        user = ratings[ratings['userId'] == user_id]
        watched_movies = set(user['movieId'])
        unwatched_movies = all_movie_ids - watched_movies
        user_to_unwatched_movies[user_id] = unwatched_movies

    return user_to_unwatched_movies

def recommend_movies(mf_model, user_to_idx, movie_to_idx, n=10):
    unwatched = unwatched_movies(train_data)
    recommendations = {}

    for user_id in user_to_idx.keys():
        user_idx = user_to_idx[user_id]
        predictions = mf_model.full_matrix()[user_idx]
        recommended_movie_idxs = np.argsort(predictions)[::-1]
        recommended_movies = [k for k, v in movie_to_idx.items() if v in recommended_movie_idxs and k in unwatched[user_id]][:n]
        recommendations[user_id] = recommended_movies

    return recommendations

def enhanced_recommend_movies(mf_model, user_to_idx, movie_to_idx, train_data, n=10, 
                           rating_threshold=3.5, min_ratings=10):
    """
    Enhanced recommendation function with multiple improvements:
    1. Considers movie popularity and rating distribution
    2. Filters out movies with too few ratings
    3. Uses rating threshold to determine "liked" movies
    4. Implements candidate selection
    """
    # Calculate movie statistics
    movie_stats = train_data.groupby('movieId').agg({
        'rating': ['count', 'mean', 'std']
    }).droplevel(0, axis=1)
    
    # Filter movies with enough ratings
    valid_movies = set(movie_stats[movie_stats['count'] >= min_ratings].index)
    
    # Calculate user preferences
    user_preferences = {}
    for user_id in train_data['userId'].unique():
        user_ratings = train_data[train_data['userId'] == user_id]
        liked_movies = user_ratings[user_ratings['rating'] >= rating_threshold]['movieId']
        
        if len(liked_movies) > 0:
            # Get genres or other features of liked movies
            user_preferences[user_id] = liked_movies.tolist()
    
    unwatched = unwatched_movies(train_data)
    recommendations = {}
    
    for user_id in user_to_idx.keys():
        if user_id not in user_preferences:
            continue
            
        user_idx = user_to_idx[user_id]
        user_unwatched = unwatched[user_id].intersection(valid_movies)
        
        if not user_unwatched:
            continue
            
        # Get predictions for unwatched movies
        predictions = []
        for movie_id in user_unwatched:
            if movie_id in movie_to_idx:
                movie_idx = movie_to_idx[movie_id]
                pred_rating = mf_model.get_rating(user_idx, movie_idx)
                
                # Calculate confidence score based on number of ratings
                confidence = 1 - (1 / (1 + movie_stats.loc[movie_id, 'count'] / 100))
                
                # Calculate popularity score normalized between 0 and 1
                popularity = movie_stats.loc[movie_id, 'count'] / movie_stats['count'].max()
                
                # Combine prediction with confidence and popularity
                adjusted_score = (0.7 * pred_rating + 
                                0.2 * confidence + 
                                0.1 * popularity)
                
                predictions.append((movie_id, adjusted_score))
        
        # Sort by adjusted score and get top N
        recommendations[user_id] = [
            movie_id for movie_id, _ in 
            sorted(predictions, key=lambda x: x[1], reverse=True)[:n]
        ]
    
    return recommendations

def adaptive_recommend_movies(mf_model, user_to_idx, movie_to_idx, train_data, n=10, 
                           min_ratings=15, rating_weight=0.5, confidence_weight=0.35,
                           similarity_weight=0.2):
    """
    Further enhanced recommendation function with:
    1. User-specific rating thresholds
    2. Genre/rating similarity matching
    3. Personalized weighting based on user behavior
    4. More sophisticated confidence scoring
    """
    # Calculate movie statistics
    movie_stats = train_data.groupby('movieId').agg({
        'rating': ['count', 'mean', 'std']
    }).droplevel(0, axis=1)
    
    # Filter movies with enough ratings
    valid_movies = set(movie_stats[movie_stats['count'] >= min_ratings].index)
    
    # Calculate user-specific rating thresholds
    user_thresholds = {}
    user_avg_ratings = {}
    for user_id in train_data['userId'].unique():
        user_ratings = train_data[train_data['userId'] == user_id]
        if len(user_ratings) >= 5:  # Ensure enough ratings to calculate threshold
            mean_rating = user_ratings['rating'].mean()
            std_rating = user_ratings['rating'].std()
            # Set threshold at mean + 0.5 * std to capture above-average ratings
            user_thresholds[user_id] = mean_rating + 0.5 * std_rating
            user_avg_ratings[user_id] = mean_rating
    
    # Calculate user preferences
    user_preferences = {}
    for user_id in user_thresholds.keys():
        user_ratings = train_data[train_data['userId'] == user_id]
        threshold = user_thresholds[user_id]
        liked_movies = user_ratings[user_ratings['rating'] >= threshold]['movieId']
        
        if len(liked_movies) > 0:
            user_preferences[user_id] = liked_movies.tolist()
    
    # Calculate movie rating patterns
    movie_patterns = {}
    for movie_id in valid_movies:
        movie_ratings = train_data[train_data['movieId'] == movie_id]
        if len(movie_ratings) >= min_ratings:
            movie_patterns[movie_id] = {
                'mean': movie_ratings['rating'].mean(),
                'std': movie_ratings['rating'].std(),
                'rating_dist': movie_ratings['rating'].value_counts().to_dict()
            }
    
    unwatched = unwatched_movies(train_data)
    recommendations = {}
    
    for user_id in user_to_idx.keys():
        if user_id not in user_preferences or user_id not in user_thresholds:
            continue
            
        user_idx = user_to_idx[user_id]
        user_unwatched = unwatched[user_id].intersection(valid_movies)
        user_threshold = user_thresholds[user_id]
        user_avg = user_avg_ratings[user_id]
        
        if not user_unwatched:
            continue
        
        # Get user's rating pattern
        user_ratings = train_data[train_data['userId'] == user_id]
        user_rating_dist = user_ratings['rating'].value_counts().to_dict()
        
        # Get predictions for unwatched movies
        predictions = []
        for movie_id in user_unwatched:
            if movie_id in movie_to_idx and movie_id in movie_patterns:
                movie_idx = movie_to_idx[movie_id]
                pred_rating = mf_model.get_rating(user_idx, movie_idx)
                
                # Calculate rating similarity score
                movie_pattern = movie_patterns[movie_id]
                rating_diff = abs(movie_pattern['mean'] - user_avg)
                rating_sim = 1 / (1 + rating_diff)
                
                # Enhanced confidence score
                n_ratings = movie_stats.loc[movie_id, 'count']
                confidence = 1 - (1 / (1 + n_ratings / 50))  # Adjusted scale
                
                # Calculate rating distribution similarity
                dist_similarity = 0
                total_ratings = sum(movie_pattern['rating_dist'].values())
                for rating, count in movie_pattern['rating_dist'].items():
                    expected = count / total_ratings
                    actual = user_rating_dist.get(rating, 0) / len(user_ratings)
                    dist_similarity += 1 - abs(expected - actual)
                dist_similarity /= 5  # Normalize
                
                # Normalize predicted rating to 0-1 scale
                norm_pred_rating = (pred_rating - 1) / 4  # Assuming 1-5 scale
                
                # Combined score with adjusted weights
                adjusted_score = (
                    rating_weight * norm_pred_rating +
                    confidence_weight * confidence +
                    similarity_weight * (0.7 * rating_sim + 0.3 * dist_similarity)
                )
                
                predictions.append((movie_id, adjusted_score))
        
        # Sort by adjusted score and get top N
        recommendations[user_id] = [
            movie_id for movie_id, _ in 
            sorted(predictions, key=lambda x: x[1], reverse=True)[:n]
        ]
    
    return recommendations

top_10_recommendations = adaptive_recommend_movies(mf, user_to_idx, movie_to_idx, train_data, n=10)

top_10_recommendations

{235: [34, 858, 2571, 260, 1270, 1221, 2858, 50, 1198, 1097],
 599: [110, 589, 3578, 1527, 1265, 527, 50, 1234, 4973, 750],
 456: [858, 527, 2858, 593, 296, 79132, 1198, 1136, 2571, 2762],
 376: [110, 318, 356, 260, 1198, 858, 58559, 1196, 1704, 5952],
 95: [527, 593, 296, 858, 541, 4993, 1136, 7153, 50, 110],
 39: [5952, 296, 4993, 2571, 593, 4973, 2959, 1200, 924, 1196],
 551: [150, 648, 480, 339, 54286, 539, 1380, 597, 780, 364],
 378: [260, 2571, 1136, 1270, 1198, 2858, 356, 1210, 589, 110],
 444: [2858, 296, 4993, 32, 1206, 7153, 2571, 858, 50, 2959],
 17: [593, 318, 110, 3578, 912, 2762, 1197, 32, 7361, 2324],
 603: [7153, 4993, 2028, 5952, 2599, 7361, 79132, 4973, 2997, 163],
 520: [1265, 593, 1036, 1198, 595, 91529, 457, 1387, 527, 89745],
 132: [110, 1682, 480, 527, 50, 4878, 780, 2762, 924, 1214],
 111: [4973, 4979, 733, 2427, 377, 2174, 2797, 648, 2997, 4014],
 198: [527, 858, 58559, 63082, 1784, 588, 1748, 3147, 457, 134853],
 327: [1198, 2706, 1210, 1291, 2571, 589, 2628, 

In [71]:
def dcg(recommended_movies, actual_movies):
    dcg_value = 0.0
    for i, movie in enumerate(recommended_movies):
        if movie in actual_movies:
            dcg_value += 1 / np.log2(i + 2)  
    return dcg_value

def idcg(actual_movies, length):
    idcg_value = 0.0
    for i in range(min(len(actual_movies), length)):
        idcg_value += 1 / np.log2(i + 2)
    return idcg_value

def recommendation_performance(recommendations, test_data):
    running_precision, running_recall, running_ndcg = 0, 0, 0

    for user_id, recommended_movies in recommendations.items():
        actual_movies = test_data[test_data['userId'] == user_id]['movieId']

        intersection = len(set(recommended_movies) & set(actual_movies))
        precision = (intersection / len(recommended_movies)) * 100
        recall = (intersection / len(actual_movies)) * 100

        running_precision += precision
        running_recall += recall

        dcg_value = dcg(recommended_movies, actual_movies)
        idcg_value = idcg(actual_movies, len(recommended_movies))
        ndcg = (dcg_value / idcg_value) * 100 if idcg_value > 0 else 0
        running_ndcg += ndcg

    precision = running_precision / len(recommendations)
    recall = running_recall / len(recommendations)
    f_measure = 2 * (precision * recall) / (precision + recall)
    ndcg = running_ndcg / len(recommendations)

    return precision, recall, f_measure, ndcg

print("Precision \t%.4f%%\nRecall \t\t%.4f%%\nF-Measure \t%.4f%%\nNDCG \t\t%.4f" % recommendation_performance(top_10_recommendations, test_data))

Precision 	14.4371%
Recall 		6.0057%
F-Measure 	8.4827%
NDCG 		0.0105
